In [1]:
#先引入后面可能用到的包（package）
import csv
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader
import backtrader as bt
import statsmodels.api as sm
import random
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [2]:
class MyStrategy(bt.Strategy):
    params=(
            ('stopup', 0.22), 
            ('stopdown', 0.08), 
            ('maperiod',15),
            ('printlog',True),
            ("lookback", 20),
            ('RSRS_period', 18),
            ('RSRS_avg_period', 600),
            )
    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_list = []

    def downcast(self, amount, lot):
        return abs(amount//lot*lot)
    
    def start(self):
        # 记录数据到本地 mark
        self.bt_file = open("results/my_strategy_backtrader_stats.csv", "w")
        self.bt_stats = csv.writer(self.bt_file)
        self.bt_stats.writerow(['datetime',
                               'drawdown', 'maxdrawdown', 
                               'timereturn',
                               'value', 'cash'])
        self.exc_file = open('results/my_strategy_exchange_record.txt', 'w')
        self.bs_stats_dic = {}

    #策略核心，根据条件执行买卖交易指令（必选）
    def next(self):
        # 记录收盘价
        if self.order: # 检查是否有指令等待执行, 
            return
        # 记录数据到本地 mark
        self.bt_stats.writerow([self.data.datetime.date(-1).strftime('%Y-%m-%d'),
                               '%.4f' % self.stats.drawdown.drawdown[0],
                               '%.4f' % self.stats.drawdown.maxdrawdown[0],
                               '%.4f' % self.stats.timereturn.line[0], # 获取当前时刻前一天的收益
                               '%.4f' % self.stats.broker.value[0], # 当前时点的前一天的总资产
                               '%.4f' % self.stats.broker.cash[0]])  # 当前时点的前一天的可用现金
        # 检查是否持仓
        self.log(f'{self.broker.getvalue():.2f}, {[(x, self.getpositionbyname(x).size) for x in self.buy_list]}')
        ### 计算股票池的动量（过去20日收盘价斜率）
        flag = False
        slope_period = []
        for code in set(self.getdatanames()):
            data = self.getdatabyname(code)
            closes_period = data.close.get(ago=-1, size=20).tolist()
            if len(closes_period) >= 20:
                flag = True
                slope = np.polyfit(range(len(closes_period)), closes_period, 1)[0]
            else:
                slope = float('inf')
            slope_period.append((code, slope))
        if flag:
            slope_period = sorted(slope_period, key=lambda x: x[1])
            slope_period = slope_period[::-1]
            print(slope_period)
            trade_codes = [x[0] for x in slope_period[:int(len(slope_period)*0.6)]]
        else:
            trade_codes = self.getdatanames()
        ###
        if len(self.buy_list) < 2:
            for code in set(trade_codes) - set(self.buy_list):
            # for code in set(self.getdatanames()) - set(self.buy_list):
                data = self.getdatabyname(code)
                price = data.lastclose
                price_up = price*(1 + self.params.stopup) # 止盈价
                price_down = price*(1-self.params.stopdown) # 止损价
                if data.predict[0] > 0.5:
                    order_value = self.broker.getvalue()*0.33
                    order_amount = self.downcast(order_value/data.close[0], 100)
                    self.order = self.buy_bracket(data, size=order_amount, name=code, limitprice = price_up, stopprice = price_down, exectype = bt.Order.Market)
                    # self.order = self.buy(data, size=order_amount, name=code)
                    self.log(f"买{code}, price:{data.close[0]:.2f}, amout: {order_amount}")
                    self.buy_list.append(code)
        else:
        # elif self.position:
            now_list = []
            for code in self.buy_list:
                data = self.getdatabyname(code)
                if data.predict[0] < 0.5:
                    self.order = self.order_target_percent(data, 0, name=code)
                    self.log(f"卖{code}, price:{data.close[0]:.2f}, pct: 0")
                    continue
                now_list.append(code)
            self.buy_list = now_list.copy()

            
    #交易记录日志（可省略，默认不输出结果）
    def log(self, txt, dt=None,doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')
            self.exc_file.write(f'{dt.isoformat()},{txt}' + '\n')

    #记录交易执行情况（可省略，默认不输出结果）
    def notify_order(self, order):
        # 如果order为submitted/accepted,返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果order为buy/sell executed,报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入:\ncode:{order.info["name"]},\
                价格:{order.executed.price},\
                成本:{order.executed.value},\
                手续费:{order.executed.comm}')
                # self.buyprice = order.executed.price
                # self.buycomm = order.executed.comm
                pre_date = self.datetime.date(-1).strftime('%Y-%m-%d')
                if  pre_date not in self.bs_stats_dic:
                    self.bs_stats_dic[pre_date] = []
                self.bs_stats_dic[pre_date].append(('buy', f'{order.info["name"]}'))
            elif order.issell():
                self.log(f'卖出:\ncode:{order.info["name"]},\
                价格：{order.executed.price},\
                成本: {order.executed.value},\
                手续费{order.executed.comm}')
                pre_date = self.datetime.date(-1).strftime('%Y-%m-%d')
                if  pre_date not in self.bs_stats_dic:
                    self.bs_stats_dic[pre_date] = []
                self.bs_stats_dic[pre_date].append(('sell', f'{order.info["name"]}'))
                
            self.bar_executed = len(self)

        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None

    #记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益：\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')

    #回测结束后输出结果（可省略，默认输出结果）
    def stop(self):
        self.log('(MA均线： %2d日) 期末总资金 %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)
        
        # 记录数据到本地 mark
        self.bt_stats.writerow([self.data.datetime.date(0).strftime('%Y-%m-%d'),
                               '%.4f' % self.stats.drawdown.drawdown[0],
                               '%.4f' % self.stats.drawdown.maxdrawdown[0],
                               '%.4f' % self.stats.timereturn.line[0],
                               '%.4f' % self.stats.broker.value[0],
                               '%.4f' % self.stats.broker.cash[0]])
        self.bt_file.close()
        self.exc_file.close()
        bs_df = pd.DataFrame()
        bs_df['datetime'] = list(self.bs_stats_dic.keys())
        bs_df['buysell'] = list(self.bs_stats_dic.values())
        bs_df.to_csv('results/my_strategy_buysell_stats.csv', index=0)
        
        

In [3]:
class MyPandasData(backtrader.feeds.PandasData): 
    lines = ('lastclose', 'predict')
    params = (
        ('lastclose', 7),
        ('predict', 8)
    )

In [4]:
base_df = pd.read_csv('../dataset/all_codes_before20200630_filtered.csv', converters={u'code':str})
all_df_record = []
for i in range(22):
    item = f'stock_lstm_v{i+1}'
    f = open(f'output/{item}_record.txt', 'r')
    lines = f.readlines()
    codes = []
    min_valid_losses, max_valid_aucs = [],  []
    min_test_losses, max_test_aucs = [], []
    for line in lines[1:]:
        line = line.strip()
        code = line.split('\t')[0]
        if code in base_df['code'].tolist():   # 688383(不在score表里面), 688386(不在score表里面), 002862(分数为0.62)
            codes.append(code)
            min_valid_losses.append(float(line.split('\t')[1]))
            max_valid_aucs.append(float(line.split('\t')[2]))
            min_test_losses.append(float(line.split('\t')[3]))
            max_test_aucs.append(float(line.split('\t')[4]))
    df_record = pd.DataFrame()
    df_record['code'] = codes
    df_record['min_valid_loss'] = min_valid_losses
    df_record['max_valid_auc'] = max_valid_aucs
    df_record['min_test_loss'] = min_test_losses
    df_record['max_test_auc'] = max_test_aucs
    all_df_record.append(df_record)
all_df_record = pd.concat(all_df_record, 0)
# all_df_record.sort_values(by='min_valid_loss', inplace=True)
all_df_record.sort_values(by='max_valid_auc', ascending=False, inplace=True)
all_df_record.reset_index(drop=True, inplace=True)

In [5]:
all_df_record

,code,min_valid_loss,max_valid_auc,min_test_loss,max_test_auc
0,001212,0.3130,1.0,0.3506,0.4058
1,301083,0.3302,1.0,0.4510,0.4072
2,301169,0.3028,1.0,0.4432,0.5431
3,301091,0.3085,1.0,0.3275,0.6512
4,300964,0.3426,1.0,0.3458,0.5276
...,...,...,...,...,...
3799,301042,0.3523,0.0,0.3456,0.5533
3800,301088,0.3332,0.0,0.3883,0.5398
3801,688701,0.3455,0.0,0.3447,0.5897
3802,688786,0.3523,0.0,0.3461,0.4893


In [6]:
# 初始化cerebro回测系统设置   
cerebro = bt.Cerebro(stdstats=False)
for code in all_df_record['code'][:50]:
    df_stock = pd.read_csv('../dataset/stock/%s.csv' % code)
    df_pred = pd.read_csv(f'output/stock_lstm_all/%s.csv' % code)
    df = pd.merge(df_stock, df_pred, how='inner', on='time')
    df.rename(columns={'chengjiaogushu':'volume', 'time':'datetime'}, inplace=True)
    df = df[['datetime', 'open', 'close', 'high', 'low', 'volume', 'avg', 'lastclose', 'predict']]
    df.index=pd.to_datetime(df.datetime)
    data = MyPandasData(dataname=df, fromdate=datetime(2022, 6, 1), todate=datetime(2022, 12, 14))   
    # 加载数据
    cerebro.adddata(data, name=code)

In [7]:
# 将交易策略加载到回测系统中
cerebro.addstrategy(MyStrategy) 
# 设置初始资本为100,000
cerebro.broker.setcash(100000.0) 
# #每次固定交易数量
# cerebro.addsizer(bt.sizers.FixedSize, stake=1000) 
#手续费
cerebro.broker.setcommission(commission=0.00025) 

In [8]:
print('初始资金: %.2f' % cerebro.broker.getvalue())
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='AnnualReturn')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = 'SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DW')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.TimeReturn)
cerebro.addobserver(bt.observers.DrawDown)
results = cerebro.run()
strat = results[0]
print('最终资金: %.2f' % cerebro.broker.getvalue())
print('年化收益率:', strat.analyzers.AnnualReturn.get_analysis())
print('夏普比率:', strat.analyzers.SharpeRatio.get_analysis())
print('回撤指标:', strat.analyzers.DW.get_analysis())

初始资金: 100000.00
2022-06-01,100000.00, []
2022-06-01,买001209, price:20.90, amout: 1500.0
2022-06-01,买688778, price:90.65, amout: 300.0
2022-06-01,买688499, price:175.50, amout: 100.0
2022-06-01,买301013, price:19.74, amout: 1600.0
2022-06-01,买688509, price:5.62, amout: 5800.0
2022-06-01,买301087, price:36.28, amout: 900.0
2022-06-01,买688626, price:32.68, amout: 1000.0
2022-06-01,买301091, price:27.60, amout: 1100.0
2022-06-01,买688360, price:25.61, amout: 1200.0
2022-06-01,买300935, price:29.86, amout: 1100.0
2022-06-01,买688728, price:19.55, amout: 1600.0
2022-06-01,买688707, price:48.65, amout: 600.0
2022-06-01,买001213, price:5.33, amout: 6100.0
2022-06-01,买000886, price:5.61, amout: 5800.0
2022-06-01,买688260, price:14.78, amout: 2200.0
2022-06-01,买002030, price:17.60, amout: 1800.0
2022-06-01,买603171, price:26.43, amout: 1200.0
2022-06-01,买300875, price:35.70, amout: 900.0
2022-06-01,买301035, price:72.98, amout: 400.0
2022-06-01,买301024, price:39.68, amout: 800.0
2022-06-01,买002531, price:13

In [9]:
portfolio_stats = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)
returns.to_csv('results/my_strategy_returns.csv')
returns

index
2022-06-01    0.000000
2022-06-02    0.017941
2022-06-06    0.027298
2022-06-07    0.010070
2022-06-08   -0.006319
                ...   
2022-12-06    0.036099
2022-12-07   -0.033659
2022-12-08   -0.016228
2022-12-09    0.025470
2022-12-12    0.040476
Name: return, Length: 132, dtype: float64

In [10]:
sum(returns)

0.27056836855641697